# Module 4: Community Detection Algorithms

Community formation is common in all types of networks, and identifying them is essential for evaluating group behavior and emergent phenomena. The general principle in finding communities is that its members will have more relationships within the group than with nodes outside their group. Identifying these related sets reveals clusters of nodes, isolated groups, and network structure.

In this notebook we'll learn how to use these algorithms in Spark and Neo4j. Before we get started let's import those libraries:

In [ ]:
from pyspark.sql.types import *
from graphframes import *
from neo4j import GraphDatabase
from pyspark.sql import functions as F
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

import pandas as pd

## Connect to Spark and Neo4j

Let's create connections to Spark and Neo4j. The following code will create a SparkContext that we'll use to connect to Spark:

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

And let's create a connection to the database. 

In [ ]:
user = "neo4j"
password = "neo"
driver = GraphDatabase.driver("bolt://data-science-training-neo4j", auth=(user, password))

## The Software Dependency Graph

Dependency graphs are particularly well suited for demonstrating the sometimes subtle differences between community detection algorithms because they tend to be more connected and hierarchical.


### Importing the Data into Apache Spark

In [ ]:
def create_software_graph():
    nodes = spark.read.csv("data/sw-nodes.csv", header=True)
    relationships = spark.read.csv("data/sw-relationships.csv", header=True)
    return GraphFrame(nodes, relationships)

In [ ]:
g = create_software_graph()

### Importing the Data into Neo4j

In [ ]:
user = "neo4j"
password = "neo"
driver = GraphDatabase.driver("bolt://localhost", auth=(user, password))

In [ ]:
with driver.session() as session:
    result = session.run("""
    LOAD CSV WITH HEADERS FROM "file:///sw-nodes.csv" AS row
    MERGE (:Library {id: row.id})
    """)
    display(result.summary().counters)
    
    result = session.run("""
    LOAD CSV WITH HEADERS FROM "file:///sw-relationships.csv" AS row
    MATCH (source:Library {id: row.src})
    MATCH (destination:Library {id: row.dst})
    MERGE (source)-[:DEPENDS_ON]->(destination)
    """)
    display(result.summary().counters)

## Triangle Count

Triangle Count determines the number of triangles passing through each node in the graph. A triangle is a set of three nodes,where each node has a relationship to all other nodes.

In [ ]:
result = g.triangleCount()
(result.sort("count", ascending=False)
.filter('count > 0')
.show())

A triangle in this graph would indicate that two of a node’s neighbors are also neighbors. Six of our libraries participate in such triangles.

What if we want to know which nodes are in those triangles? That’s where a triangle stream comes in. 

In [ ]:
query = """
CALL algo.triangle.stream("Library","DEPENDS_ON")
YIELD nodeA, nodeB, nodeC
RETURN algo.getNodeById(nodeA).id AS nodeA,
       algo.getNodeById(nodeB).id AS nodeB,
       algo.getNodeById(nodeC).id AS nodeC
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

We can see these triangles in the diagram below:

![](images/triangles.svg)

The Clustering Coefficient algorithm measures how tightly a group is clustered compared to how tightly it could be clustered. The algorithm uses Triangle Count in its calculations, which provides a ratio of existing triangles to possible relationships.

In [ ]:
query = """
CALL algo.triangleCount.stream('Library', 'DEPENDS_ON')
YIELD nodeId, triangles, coefficient
WHERE coefficient > 0
RETURN algo.getNodeById(nodeId).id AS library, coefficient
ORDER BY coefficient DESC
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

ipykernel has a score of 1, which means that all ipykernel’s neighbors are neighbors of each other.

## Strongly Connected Components

The Strongly Connected Components (SCC) algorithm is one of the earliest graph algorithms. SCC finds sets of connected nodes in a directed graph where each node is reachable in both directions from any other node in the same set.

In [ ]:
result = g.stronglyConnectedComponents(maxIter=10)
(result.sort("component")
 .groupby("component")
 .agg(F.collect_list("id").alias("libraries"))
 .show(truncate=False))

In [ ]:
query = """
CALL algo.scc.stream("Library", "DEPENDS_ON")
YIELD nodeId, partition
RETURN partition, collect(algo.getNodeById(nodeId).id) AS libraries
ORDER BY size(libraries) DESC
"""


with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

## Connected Components

The Connected Components algorithm (sometimes called Union Find or Weakly Connected Components) finds sets of connected nodes in an undirected graph where each node is reachable from any other node in the same set.

In [ ]:
# sc.setCheckpointDir("/tmp")
result = g.connectedComponents()
(result.sort("component")
 .groupby("component")
 .agg(F.collect_list("id").alias("libraries"))
 .show(truncate=False))

In [ ]:
query = """
CALL algo.unionFind.stream("Library", "DEPENDS_ON")
YIELD nodeId,setId
RETURN setId, collect(algo.getNodeById(nodeId).id) AS libraries
ORDER BY size(libraries) DESC
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

## Label Propagation

The Label Propagation algorithm (LPA) is a fast algorithm for finding communities in a graph. In LPA, nodes select their group based on their direct neighbors. This process is well suited to networks where groupings are less clear and weights can be used to help a node determine which community to place itself within.

In [ ]:
result = g.labelPropagation(maxIter=10)
(result
 .sort("label")
 .groupby("label")
 .agg(F.collect_list("id"))
 .show(truncate=False))

Compared to Connected Components, we have more clusters of libraries in this example. LPA is less strict than Connected Components with respect to how it determines clusters. Two neighbors (directly connected nodes) may be found to be in different clusters using Label Propagation.

In [ ]:
query = """
CALL algo.labelPropagation.stream("Library", "DEPENDS_ON", { 
 iterations: 10 
})
YIELD nodeId, label
RETURN label, 
       collect(algo.getNodeById(nodeId).id) AS libraries
ORDER BY size(libraries) DESC
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

## Louvain Modularity

The Louvain Modularity algorithm finds clusters by comparing community density as it assigns nodes to different groups. You can think of this as a “what if ” analysis to try various groupings with the goal of reaching a global optimum.

In [ ]:
query = """
CALL algo.louvain.stream("Library", "DEPENDS_ON", {
  includeIntermediateCommunities: true
})
YIELD nodeId, communities, community
RETURN algo.getNodeById(nodeId).id AS libraries, community, communities
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)

In [ ]:
query = """
CALL algo.louvain("Library", "DEPENDS_ON", {
  writeProperty: 'community',
  includeIntermediateCommunities: true,
  intermediateCommunitiesWriteProperty: 'communities'
})
"""

with driver.session() as session:
    result = session.run(query)
    display(result.summary().counters)

In [ ]:
query = """
MATCH (l:Library)
RETURN l.communities[-1] AS community, collect(l.id) AS libraries
ORDER BY size(libraries) DESC
"""

with driver.session() as session:
    rows = session.run(query)
    df = pd.DataFrame([dict(record) for record in rows])

display(df)